## Reserve and configure resources on Chameleon

## Create lease and reserve resources


In [ ]:
import json
import os
import chi
import chi.ssh
import chi.network
from datetime import datetime, timedelta
from dateutil import tz

In [ ]:
#Config with your project and site
project = os.getenv("OS_PROJECT_NAME")
chi.use_site('CHI@UC')
chi.set('project_name', project)

In [ ]:
# Name resources with your username for easier identification
username = os.getenv("USER")
prefix = username + "_k8s-ml_"
server_name = prefix + 'Server'
network_name = prefix + 'Net'
subnet_name = prefix + 'Subnet'
router_name = prefix + 'Router'
lease_name = prefix + 'Lease'

In [ ]:
#Server attributes
image_name='CC-Ubuntu18.04'
node_type="compute_cascadelake_r"
server_count=3
physical_network="physnet1"
subnet_cidr = '192.168.1.0/24'

#### Create the Network

In [ ]:
# Create a network out of provider network physnet1
network = chi.network.create_network(network_name, provider='physnet1')
network_id = network['id']

print(f'Network ID: {network_id}')

#### Configure the Network


In [ ]:
subnet = chi.network.create_subnet(subnet_name, network_id, cidr=subnet_cidr, gateway_ip=None)
router = chi.network.create_router(router_name, gw_network_name='public')
chi.network.add_subnet_to_router(router['id'], subnet['id'])

#### Create a lease with network and 3 servers

In [ ]:
import chi.lease

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=3)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
chi.lease.add_node_reservation(reservation_list, count=server_count, node_type=node_type)
chi.lease.add_fip_reservation(reservation_list, count=server_count)

# Create the lease
lease = chi.lease.create_lease(lease_name, 
                               start_date=start_date,
                               end_date=end_date,
                               reservations=reservation_list)

lease_id = lease["id"]

chi.lease.wait_for_active(lease_id)

#### Get the Reservations

In [ ]:
compute_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
floatingip_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'virtual:floatingip'][0]['id']

print(f"compute_reservation_id: {compute_reservation_id}")
print(f"floatingip_reservation_id: {floatingip_reservation_id}")

#### Start the Server

In [ ]:
#create the server
servers = []
for i in range(server_count):
    servers.append(chi.server.create_server(server_name+f"-{i}", 
                                  compute_reservation_id,
                                  network_id=network_id,
                                  nics=[{"net-id": network_id,"v4-fixed-ip": f"192.168.1.1{i}" }],
                                  image_name=image_name,
                                  count=1))


#### Associate the floating IPs

In [ ]:
nodes = {}

for j,i in enumerate(servers):
    chi.server.wait_for_active(i.id)
    floating_ip = chi.server.associate_floating_ip(i.id)
    nodes[j] = floating_ip

    print(f'Floating IP of node_{j}: {floating_ip}')

#### Wait for SSH access to all servers

In [ ]:
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)

### Configuring the servers

#### Install keys for SSH access between all servers

In [ ]:
for node_ip in nodes.values():
    remote = chi.ssh.Remote(node_ip)
    remote.run('ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa -q -N ""')
    public_key = remote.run('cat ~/.ssh/id_rsa.pub').tail("stdout")[2:]
    public_key = public_key.replace("\n", "")
    for other_node_ip in nodes.values():
        remote = chi.ssh.Remote(other_node_ip)
        remote.run(f'echo {public_key} >> ~/.ssh/authorized_keys') 

#### Store the IP address of each host

When creating the cluster, we will need these addresses, which are assigned by DHCP and so are different each time.

In [ ]:
physical_ips=[]
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)
    remote = chi.ssh.Remote(node_ip)
    physical_ips.append(remote.run('hostname -I').tail("stdout")[2:-1])

#### Use Kubespray to prepare a Kubernetes cluster

In [ ]:
ansible_node = nodes[0]
remote = chi.ssh.Remote(ansible_node)
remote.run("sudo apt install virtualenv")
remote.run("virtualenv -p python3 myenv")
remote.run("git clone https://github.com/kubernetes-sigs/kubespray.git")
remote.run("cd kubespray; git checkout release-2.16")
remote.run("source myenv/bin/activate; cd kubespray; pip3 install -r requirements.txt")
remote.run("cd kubespray; cp -rfp inventory/sample inventory/mycluster")
remote.run("rm -rf kubespray/contrib/inventory_builder/inventory.py")
remote.run("rm -rf kubespray/inventory/mycluster/group_vars/k8s_cluster/addons.yml")
remote.run("wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/inventory.py -O kubespray/contrib/inventory_builder/inventory.py")
remote.run("wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/addons.yml -O kubespray/inventory/mycluster/group_vars/k8s_cluster/addons.yml")
remote.run(f"source myenv/bin/activate; declare -a IPS=({physical_ips[0]} {physical_ips[1]} {physical_ips[2]});"+"cd kubespray; CONFIG_FILE=inventory/mycluster/hosts.yaml python3 contrib/inventory_builder/inventory.py ${IPS[@]}")

In [ ]:
# need a specific versino of pyOpenSSL
remote.run("source myenv/bin/activate; sudo rm -rf /usr/local/lib/python3.6/dist-packages/OpenSSL")
remote.run("source myenv/bin/activate; sudo rm -rf /usr/local/lib/python3.6/dist-packages/pyOpenSSL-22.1.0.dist-info/")
remote.run("source myenv/bin/activate; sudo pip3 install pyOpenSSL==22.0.0")

In [ ]:
remote.run("source myenv/bin/activate; cd kubespray; ansible-playbook -i inventory/mycluster/hosts.yaml  --become --become-user=root cluster.yml")

In [ ]:
# give cc user the kube config file
remote.run("sudo cp -R /root/.kube /home/cc/.kube; sudo chown -R cc /home/cc/.kube; sudo chgrp -R cc /home/cc/.kube")

In [ ]:
# enable external access to the port on which we will deploy the service
for i in nodes.keys():
    remote = chi.ssh.Remote(nodes[i])
    remote.run("sudo ufw allow 32000")

In [ ]:
# also make sure docker can run without sudo 
# start a new SSH session after changing group membership
remote = chi.ssh.Remote(nodes[0])
remote.run("sudo groupadd docker; sudo usermod -aG docker $USER")
remote = chi.ssh.Remote(nodes[0])

In [ ]:
# copy docker configuration for private registry
remote.run("docker run -d -p 5000:5000 --restart always --name registry registry:2")
remote.run("sudo wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/daemon.json -O /etc/docker/daemon.json")
remote.run("sudo service docker restart")

#### Validate your setup

Use the following command to validate your Kubernetes cluster with 3 servers:

In [ ]:
remote.run("kubectl get nodes")

Also test the Docker install:

In [ ]:
remote.run("docker run hello-world")

# Delete resources

### Delete server

In [ ]:
if True: 
    for i in servers:
      chi.server.delete_server(i.id)

#### De-configure Network



In [ ]:
router_id = router['id']
subnet_id = subnet['id']

try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network_by_name error: {str(e)}")
    pass


#### Release Lease

In [ ]:
chi.lease.delete_lease(lease_id)